In [6]:
import pandas as pd
import smtplib


df = pd.read_csv("Apartments.csv", encoding="utf-16", delimiter = ";", error_bad_lines = False)
df.head()

### DEALING WITH MISSING DATA

#df[df.isnull().any(axis= 1)]

df.dropna(inplace = True)
df.reset_index(inplace = True, drop = True)

### CHECK FOR DUPLICATES

#df.duplicated().any()

### DATA ADJUSTMENTS

# removing spaces
cols = df.columns

for col in df.columns:
    df[f"{col}"] = df[f"{col}"].str.strip()


# seperate data for price per agreement
po_dogovoru = df[df["price"] == "Po dogovoru"]
po_dogovoru.reset_index(inplace = True, drop = True)


# seperate action price
akcija = df[df["price"] == "Stanje"]


# regular price
df = df[(df["price"] != "Po dogovoru") & (df["price"] != "Stanje")]
df.reset_index(inplace = True, drop = True)



### PRICE

# remove currency
df["price"] = df["price"].str[:-2].str.strip()

# remove "."
df["price"] = df["price"].str.replace(".", "")

# change price to numeric
df["price"] = pd.to_numeric(df["price"])


# N OF SQM

# taking just numbers
df["n of sqm"] = df["n of sqm"].str[:2]


# removing non num entries
numbers = [str(i) for i in range(100)]

df = df[df["n of sqm"].isin(numbers)]
df.reset_index(inplace = True, drop = True)


# change to numeric
df["n of sqm"] = pd.to_numeric(df["n of sqm"])


# create price of sqm
df["price_sqm"] = df["price"]/df["n of sqm"]


# N OF ROOMS

# recode
df["n of rooms"].replace(
    
    {"Trosoban (3)": "3",
     "Dvosoban (2)": "2",
     "Četverosoban (4)": "4",
     "Jednosoban (1)": "1",
     "Petosoban i više": "5",
     "Garsonjera" : "0",
     "Jednoiposoban (1.5)": "1.5",}, inplace = True)

# convert to num
df["n of rooms"] = pd.to_numeric(df["n of rooms"], downcast = "float")



# PUBLISH DATE

# take just date
df["publish date"] = df["publish date"].str[:-8]

# convert to date
df["publish date"] = pd.to_datetime(df["publish date"])



# setting up conditions and filtering data based on them
'''
Conditions:
- price <= 150000
- n of rooms >= 2
- n of sqm >= 30
- location Novo Sarajevo and Centar 
- floor != prizemlje & suteren
''' 
df = df[(df["price"] <= 150000) & 
    (df["n of rooms"] >= 2) & 
    (df["n of sqm"] >= 30) & 
    ((df["location"] == "Sarajevo - Centar") | (df["location"] == "Novo Sarajevo")) & 
    (df["floor"] != "Prizemlje") & 
    (df["floor"] != "Suteren")]

df = df.sort_values(by = "publish date", ascending = False)
df.reset_index(inplace = True, drop = True)

C:\Users\Amar\AppData\Local\Temp\ipykernel_2668\3199671740.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("Apartments.csv", encoding="utf-16", delimiter = ";", error_bad_lines = False)
b'Skipping line 946: expected 10 fields, saw 11\n'
C:\Users\Amar\AppData\Local\Temp\ipykernel_2668\3199671740.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["price"] = df["price"].str.replace(".", "")


In [7]:
df

,header,location,price,publish date,n of sqm,floor,n of rooms,furnished?,heating system,link,price_sqm
0,NOVOGRADNJA CENTAR! Dvosoban stan cca 44 m2! U...,Sarajevo - Centar,131850,2022-12-04,44,1,2.0,Nenamješten,Plin,https://www.olx.ba/artikal/47674177/novogradnj...,2996.590909
1,NN Nekretnine: Sprat kuće - Mejtaš - Čekaluša,Sarajevo - Centar,118000,2022-12-01,62,Visoko prizemlje,3.0,Nenamješten,Ostalo,https://www.olx.ba/artikal/46303962/nn-nekretn...,1903.225806
2,Dvosoban stan sa garažom / 3. sprat / Vraca,Novo Sarajevo,139500,2022-11-03,46,3,2.0,Namješten,Centralno (Plin),https://www.olx.ba/artikal/47203433/dvosoban-s...,3032.608696
3,"Stan sa garazom-Dobojska, Sarajevo",Novo Sarajevo,139500,2022-10-03,47,3,2.0,Namješten,Plin,https://www.olx.ba/artikal/47199444/stan-sa-ga...,2968.085106
4,Dvosoban stan novije gradnje od 45m2 u Velešićima,Novo Sarajevo,125000,2022-09-05,45,1,2.0,Namješten,Centralno (Plin),https://www.olx.ba/artikal/48042158/dvosoban-s...,2777.777778
5,NN Nekretnine: Dvosoban adaptiran stan - Bjelave,Sarajevo - Centar,148000,2022-09-05,42,3,2.0,Namješten,Plin,https://www.olx.ba/artikal/48040802/nn-nekretn...,3523.809524
6,Dvosoban renoviran stan Sarajevo Centar Zaima ...,Sarajevo - Centar,149000,2022-06-18,45,1,2.0,Namješten,Struja,https://www.olx.ba/artikal/48543663/dvosoban-r...,3311.111111
7,Stan Grbavica kod soppinga,Novo Sarajevo,135000,2022-06-17,38,4,2.0,Namješten,Centralno (gradsko),https://www.olx.ba/artikal/48526088/stan-grbav...,3552.631579
8,"Dvosoban stan Socijalno, 41m2",Novo Sarajevo,149000,2022-06-15,41,Visoko prizemlje,2.0,Namješten,Centralno (gradsko),https://www.olx.ba/artikal/48504008/dvosoban-s...,3634.146341
9,"POSJED PRODAJE:Dvosoban namješten stan, Socijalno",Novo Sarajevo,149000,2022-06-13,41,Visoko prizemlje,2.0,Namješten,Centralno (gradsko),https://www.olx.ba/artikal/48475581/posjed-pro...,3634.146341


In [ ]:
# e-mail setup
sender_email = "amar.aladzuz@outlook.com"
rec_email = "amar.aladzuz@outlook.com"
password = input(str("Please enter pass: "))
mssg = "test"

server = smtplib.SMTP("smtp-mail.outlook.com", 587)
server.starttls()


server.login(sender_email, password)
print("Login success")

server.sendmail(sender_email, rec_email, mssg)
print("Email sent")